### 한글 추출, 형용사, 동사, 명사,감탄사 추출

In [4]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt

In [2]:
df = pd.read_csv('./data/review_data_2t.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141254 entries, 0 to 141253
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   review  141254 non-null  object
 1   시설분류    141254 non-null  object
dtypes: object(2)
memory usage: 2.2+ MB


In [5]:
df.head(2)

,review,시설분류
0,?,문화
1,친절해요,문화


In [6]:
df.columns = ['review','label']

In [7]:
df.head(2)

,review,label
0,?,문화
1,친절해요,문화


In [8]:
# 정규 표현식을 사용하여 한글만 추출
def extract_korean(text):

    # 한글과 공백만 추출
    text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\s]','',text)
    # 공백 2개 이상인경우 1개로 변환
    text = re.sub(r'\s+',' ',text)

    return text

In [27]:
# 형태소 분석 및 특정 품사 추출 함수
def extract_pos(text) :
    okt = Okt()
    pos_data = okt.pos(text)
    adj = ' '.join([word for word, tag in pos_data if tag in ['Adjective']])
    verb = ' '.join([word for word, tag in pos_data if tag in ['Verb']])
    noun = ' '.join([word for word, tag in pos_data if tag in ['Noun']])
    exclamation = ' '.join([word for word, tag in pos_data if tag in ['Exclamation']])
    
    # 각 품사에 대해 값이 없는 경우 NaN 할당
    adj = adj if adj else np.nan
    verb = verb if verb else np.nan
    noun = noun if noun else np.nan
    exclamation = exclamation if exclamation else np.nan
    
    return adj, verb, noun, exclamation                        
                            

In [28]:
# 한글만 추출하여 새로운 컬럼에 저장
df['review2'] = df['review'].map(extract_korean)

In [38]:
df_test = df.head(5).copy()
display(df_test)
df_test[['adj','verb','noun','exclamation']] = df_test['review2'].apply(extract_pos).apply(pd.Series)
df_test

,review,label,review2
0,?,문화,
1,친절해요,문화,친절해요
2,좋아요 만족스러워요,문화,좋아요 만족스러워요
3,와우~~,문화,와우
4,굿,문화,굿


,review,label,review2,adj,verb,noun,exclamation
0,?,문화,,NaN,NaN,NaN,NaN
1,친절해요,문화,친절해요,친절해요,NaN,NaN,NaN
2,좋아요 만족스러워요,문화,좋아요 만족스러워요,좋아요 만족스러워요,NaN,NaN,NaN
3,와우~~,문화,와우,NaN,NaN,와우,NaN
4,굿,문화,굿,NaN,NaN,굿,NaN


In [40]:
from tqdm import tqdm
tqdm.pandas()
# 형용사,동사,명사, 감탄사 추출
df[['adj','verb','noun','exclamation']] = df['review2'].progress_apply(extract_pos).progress_apply(pd.Series)

100%|████████████████████████████████████████████████████████████████████████| 141254/141254 [00:34<00:00, 4115.44it/s]


In [41]:
df.to_csv('./data/review_data_2t_1.csv',index=False)

In [5]:
df2 = pd.read_csv('./data/review_data_2t_1.csv',encoding='utf-8')

In [8]:
df2.head(5)

,review,label,review2,adj,verb,noun,exclamation
0,?,문화,NaN,NaN,NaN,NaN,NaN
1,친절해요,문화,친절해요,친절해요,NaN,NaN,NaN
2,좋아요 만족스러워요,문화,좋아요 만족스러워요,좋아요 만족스러워요,NaN,NaN,NaN
3,와우~~,문화,와우,NaN,NaN,와우,NaN
4,굿,문화,굿,NaN,NaN,굿,NaN


In [9]:
# NaN 값제거 후 오름차순 정렬 후 공백을 구분자로 한개의 문자열로 합친후 공백을 구분자로 리스트 변환
adj_list = ' '.join(df2['adj'].dropna().sort_values()).split()
print(len(adj_list))

235178


In [10]:
# 문자열 길이가 1인 요소를 제외하고 새 리스트 생성
filtered_adj_list = [word for word in adj_list if len(word) > 1]

In [11]:
# set로 변환후 중복 제거
deduplicated_adj_list = sorted(list(set(filtered_adj_list)))

In [32]:
len(deduplicated_adj_list)

12883

In [36]:
# 어간 추출
def extract_stem(words):
    okt = Okt()
    stems = []
    for word in words:
        # 어간 추출
        stems = okt.normalize(words)
    return stems

In [38]:
from tqdm import tqdm
tqdm.pandas()
tmp = pd.Series(deduplicated_adj_list).progress_map(extract_stem)

  0%|                                                                              | 1/12883 [00:00<00:12, 1001.74it/s]


TypeError: Okt.__init__() got an unexpected keyword argument 'stems'

In [35]:
s1 = pd.Series(deduplicated_adj_list)
s2 = pd.Series(tmp)
df_adj_stem = pd.DataFrame({'adj' : s1, 'adj_stem': s2})
df_adj_stem

,adj,adj_stem
0,가까,가까
1,가까와,가까와
2,가까우,가까우
3,가까우니,가까우니
4,가까우니까,가까우니까
...,...,...
12878,힘없어서,힘없어서
12879,힘없이,힘없이
12880,힘차,힘차
12881,힘차게,힘차게


In [26]:
len(tmp)

12883

In [31]:
# 중복괸 항목 찾기
duplicates = tmp[tmp.duplicated()]
# 중복된 값 출력
print("중복된 값들:", duplicates.unique())

중복된 값들: ['가능하고요' '가능합니다' '가득하세요' '간편하고요' '같더라고요' '건강하더라고요' '건강하세요' '건강합니다' '계셔주세요'
 '계시더라고요' '고맙더라고요' '곱니다' '괜찮더라고요' '궁금합니다' '그래도' '그래요' '그렇더라고요' '거지같은'
 '깊더라고요' '깔끔하고요' '깔끔합니다' '깨끗하고요' '깨끗합니다' '꼼꼼하세요' '꼼꼼하십니다' '넓더라고요' '넘치세요'
 '다르더라고요' '다양하고요' '다양합니다' '더라고요' '든든합니다' '딱입니다' '뛰어나십니다' '만족스럽답니다' '만족합니다'
 '많더라고요' '많았답니다' '맛있답니다' '맛있더라고요' '맛있었답니다' '멋있으세요' '멋지더라고요' '멋집니다'
 '부탁드립니다' '불친절하세요' '비싸더라고요' '뿌듯하더라고요' '뿌듯합니다' '뿌듯했답니다' '상냥합니다' '순하더라고요'
 '신기합니다' '쏠쏠합니다' '아니더라고요' '아쉽더라고요' '안녕하세요' '안되더라고요' '안됩니다' '어렵더라고요' '없답니다'
 '없더라고요' '예쁘더라고요' '예쁘세요' '예쁩니다' '용이합니다' '용합니다' '웬만한' '웬만한거' '웬만한건' '예쁘다'
 '이쁘더라고요' '이쁘십니다' '이쁩니다' '입니다' '있겠더라고요' '있답니다' '있더라고요' '있어주세요' '자세합니다'
 '재밌으세요' '저렴하고요' '저렴하더라고요' '저렴합니다' '조용하고요' '좋겠다' '좋겠어' '좋겠어여' '좋겠어요'
 '좋답니다' '좋더라고요' '좋아하고요' '좋아하더라고요' '좋아한답니다' '좋아합니다' '좋았답니다' '좋으세요' '좋으십니다'
 '죄송합니다' '착하고요' '착하세요' '착합니다' '청결하고요' '청결합니다' '추합니다' '친절하고요' '친절하더라고요'
 '친절하세요' '친절하시더라고요' '친절하십니다' '친절합니다' '편리하고요' '편리합니다' '편하고요' '편하더라고요'
 '편합니다' '행복하답니다' '행복하더라고요' '행복합니다' '행복했어요' '활발합니다' '후회됩니다'

In [28]:
# 중복 제거
len(tmp.drop_duplicates())

12758

In [39]:
deduplicated_adj_list

['가까',
 '가까와',
 '가까우',
 '가까우니',
 '가까우니까',
 '가까우니깐',
 '가까우면',
 '가까우면서',
 '가까우시다면',
 '가까우시면',
 '가까우어요',
 '가까운',
 '가까운거리라',
 '가까운건',
 '가까운게',
 '가까운대',
 '가까운데',
 '가까운데가',
 '가까운데로',
 '가까운데를',
 '가까운데서',
 '가까운데에',
 '가까운으로',
 '가까울수록',
 '가까워',
 '가까워도',
 '가까워서',
 '가까워서도',
 '가까워서요',
 '가까워요',
 '가까웠고',
 '가까웠는데',
 '가까웠습니다',
 '가까웠어요',
 '가까웠었는데',
 '가까웠으면',
 '가까웠음에도',
 '가까으면',
 '가까이있어서',
 '가까히',
 '가까히에서',
 '가깝',
 '가깝게',
 '가깝고',
 '가깝고이',
 '가깝구',
 '가깝구요',
 '가깝기',
 '가깝기만',
 '가깝길래',
 '가깝네요',
 '가깝다',
 '가깝다고',
 '가깝다는',
 '가깝다는거',
 '가깝다면',
 '가깝도',
 '가깝습니다',
 '가깝이',
 '가깝지',
 '가깝지는',
 '가깝지도',
 '가깝지만',
 '가깝지요',
 '가깝진',
 '가난한',
 '가늘게',
 '가늘고',
 '가능하',
 '가능하거나',
 '가능하게',
 '가능하게끔',
 '가능하고',
 '가능하고요',
 '가능하구',
 '가능하구요',
 '가능하기',
 '가능하네요',
 '가능하니',
 '가능하니까',
 '가능하다',
 '가능하다고',
 '가능하다구',
 '가능하다는',
 '가능하다니',
 '가능하다니깐',
 '가능하단다',
 '가능하답니다',
 '가능하더라고',
 '가능하더라구요',
 '가능하도록',
 '가능하며',
 '가능하면',
 '가능하면서',
 '가능하세요',
 '가능하시다고',
 '가능하신',
 '가능하실까',
 '가능하여',
 '가능하여서',
 '가능하죠',
 '가능하지만',
 '가능한',
 '가능한거면',
 '가능한건',
 '가능한것도',
 '가능한게'